<a href="https://colab.research.google.com/github/mitchellacohen/retrosheet/blob/main/Mets_Retrosheet_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read Mets plays from Retrosheet event data. Add the game_id to every row.

In [1]:
import pandas

YEAR = "2022"
TEAM = "NYN"

url = "https://raw.githubusercontent.com/chadwickbureau/retrosheet/master/event/regular/{}{}.EVN".format(YEAR, TEAM)
team_events = pandas.read_csv(
    url,
    names = ['col{}'.format(index) for index in range(0,7)],
    dtype = str,
    engine='python',
)

# Add the game_id to every row. The game_id is the previous col1 value from the previous row with 'id' in col0.
team_events['game_id'] = team_events['col1'].where(
    team_events['col0'] == 'id',
    None
)
# TODO: Find alternative to copy if it is a performance (mem or time) issue.
team_events['game_id'] = team_events['game_id'].ffill().copy()

from IPython import display
display.display(team_events)

,col0,col1,col2,col3,col4,col5,col6,game_id
0,id,NYN202204150,None,None,None,None,None,NYN202204150
1,version,2,None,None,None,None,None,NYN202204150
2,info,visteam,ARI,None,None,None,None,NYN202204150
3,info,hometeam,NYN,None,None,None,None,NYN202204150
4,info,site,NYC20,None,None,None,None,NYN202204150
...,...,...,...,...,...,...,...,...
12841,data,er,fedde001,9,None,None,None,NYN202210050
12842,data,er,willt002,2,None,None,None,NYN202210050
12843,data,er,rodrj004,0,None,None,None,NYN202210050
12844,data,er,may-t001,0,None,None,None,NYN202210050


Create dictionary of player ID to player name. We use player ID in case player name is not unique, but can use the dictionary to display the player name in reports.

In [2]:
starters = team_events.copy()
starters = starters[team_events['col0'] == 'start']
players = dict(zip(starters.col1, starters.col2))
print(players)

{'varsd001': 'Daulton Varsho', 'martk001': 'Ketel Marte', 'perad001': 'David Peralta', 'walkc002': 'Christian Walker', 'smitp002': 'Pavin Smith', 'beers001': 'Seth Beer', 'alcas002': 'Sergio Alcantara', 'herrj003': 'Jose Herrera', 'perdg001': 'Geraldo Perdomo', 'daviz001': 'Zach Davies', 'mcnej002': 'Jeff McNeil', 'marts002': 'Starling Marte', 'lindf001': 'Francisco Lindor', 'alonp001': 'Pete Alonso', 'escoe001': 'Eduardo Escobar', 'canor001': 'Robinson Cano', 'davij006': 'J.D. Davis', 'jankt001': 'Travis Jankowski', 'mccaj001': 'James McCann', 'bassc001': 'Chris Bassitt', 'kellc002': 'Carson Kelly', 'herny003': 'Yonny Hernandez', 'gallz001': 'Zac Gallen', 'smitd008': 'Dominic Smith', 'nidot001': 'Tomas Nido', 'carrc003': 'Carlos Carrasco', 'hummc001': 'Cooper Hummel', 'mccaj003': 'Jake McCarthy', 'casth002': 'Humberto Castellanos', 'guill001': 'Luis Guillorme', 'peted003': 'David Peterson', 'yastm001': 'Mike Yastrzemski', 'beltb001': 'Brandon Belt', 'florw001': 'Wilmer Flores', 'pedej

Pick out only the plays and rename the columns for the play type.

In [3]:
team_plays = team_events.copy()
team_plays = team_plays[team_events['col0'] == 'play']
team_plays.drop(columns=["col0"], inplace=True)
team_plays.rename(
    columns={
        'col1': 'inning',
        'col2': 'inning_half',
        'col3': 'player_id',
        'col4': 'count_on_batter',
        'col5': 'pitches',
        'col6': 'result',
    },
    inplace=True
)

from IPython import display
display.display(team_plays)

,inning,inning_half,player_id,count_on_batter,pitches,result,game_id
53,1,0,varsd001,02,CFFS,K,NYN202204150
54,1,0,martk001,11,BSX,D8/L89XD+,NYN202204150
55,1,0,perad001,32,BBCSF*BFFB,W,NYN202204150
56,1,0,walkc002,32,CSBBBS,K,NYN202204150
57,1,0,smitp002,11,CBX,6/P6M,NYN202204150
...,...,...,...,...,...,...,...
12831,9,0,vargi001,00,NaN,NP,NYN202210050
12833,9,0,vargi001,10,.BX,7/L7,NYN202210050
12834,9,0,hernc005,00,X,63/G6,NYN202210050
12835,9,0,calla002,22,CSBFBX,D7/L7L,NYN202210050


Cleanse to end result of plate appearance -- including adding 1 char results

In [4]:
team_pa_results = team_plays.copy()

# Remove non-plays.
team_pa_results = team_pa_results[team_pa_results.result.str.startswith('NP') == False]

# Remove base running events.
base_running_events = ('BK', 'CS', 'DI', 'OA', 'PB', 'WP', 'PO', 'SB')
team_pa_results = team_pa_results[~team_pa_results.result.str.startswith(base_running_events)]

# Change Hit-by-Pitch (HP) to P so we can have first char of H mean Home Run.
team_pa_results['result_char'] = team_pa_results['result'].where(
    ~team_pa_results['result'].str.startswith('HP'),
    'P'
).astype(str).str[0]

from IPython import display
display.display(team_pa_results)

,inning,inning_half,player_id,count_on_batter,pitches,result,game_id,result_char
53,1,0,varsd001,02,CFFS,K,NYN202204150,K
54,1,0,martk001,11,BSX,D8/L89XD+,NYN202204150,D
55,1,0,perad001,32,BBCSF*BFFB,W,NYN202204150,W
56,1,0,walkc002,32,CSBBBS,K,NYN202204150,K
57,1,0,smitp002,11,CBX,6/P6M,NYN202204150,6
...,...,...,...,...,...,...,...,...
12830,8,1,goret001,22,FBSBX,13/G13,NYN202210050,1
12833,9,0,vargi001,10,.BX,7/L7,NYN202210050,7
12834,9,0,hernc005,00,X,63/G6,NYN202210050,6
12835,9,0,calla002,22,CSBFBX,D7/L7L,NYN202210050,D


Add the next batter (for the same team) to each row. Note this is the actual next batter (after substitions) as opposed to who was next in the lineup. The last batter of the game for each team will have a NaN value.

In [5]:
team_pa_results['game_team'] = team_pa_results['game_id'] + team_pa_results['inning_half']

team_pa_results['next_batter'] = team_pa_results.groupby('game_team')['player_id'].shift(-1)

from IPython import display
display.display(team_pa_results)

,inning,inning_half,player_id,count_on_batter,pitches,result,game_id,result_char,game_team,next_batter
53,1,0,varsd001,02,CFFS,K,NYN202204150,K,NYN2022041500,martk001
54,1,0,martk001,11,BSX,D8/L89XD+,NYN202204150,D,NYN2022041500,perad001
55,1,0,perad001,32,BBCSF*BFFB,W,NYN202204150,W,NYN2022041500,walkc002
56,1,0,walkc002,32,CSBBBS,K,NYN202204150,K,NYN2022041500,smitp002
57,1,0,smitp002,11,CBX,6/P6M,NYN202204150,6,NYN2022041500,beers001
...,...,...,...,...,...,...,...,...,...,...
12830,8,1,goret001,22,FBSBX,13/G13,NYN202210050,1,NYN2022100501,NaN
12833,9,0,vargi001,10,.BX,7/L7,NYN202210050,7,NYN2022100500,hernc005
12834,9,0,hernc005,00,X,63/G6,NYN202210050,6,NYN2022100500,calla002
12835,9,0,calla002,22,CSBFBX,D7/L7L,NYN202210050,D,NYN2022100500,palaj001


Choose hitters with sufficient plate appearances. This also serves to filter out oppositions (which could have been done in a more elegant fashion).

In [6]:
# Half the min plate appearances to qualify for batting title.
MIN_PA = 251
hitters = team_pa_results.groupby(['player_id']).size()
hitters = hitters[hitters >= MIN_PA].index.tolist()
print('Qualifying hitters: {}\n'.format(hitters))

hitter_plays = team_pa_results[team_pa_results.player_id.isin(hitters)]

from IPython import display
display.display(hitter_plays)

Qualifying hitters: ['alonp001', 'canhm001', 'escoe001', 'lindf001', 'marts002', 'mcnej002', 'nimmb001']



,inning,inning_half,player_id,count_on_batter,pitches,result,game_id,result_char,game_team,next_batter
58,1,1,mcnej002,22,CBFBX,S5/G56,NYN202204150,S,NYN2022041501,marts002
59,1,1,marts002,10,BX,S8/F8S.1-2,NYN202204150,S,NYN2022041501,lindf001
60,1,1,lindf001,02,CFX,46(1)/FO/G34.2-3;B-1,NYN202204150,4,NYN2022041501,alonp001
61,1,1,alonp001,10,*BX,7/SF/F7LSF/FL.3-H;1-2,NYN202204150,7,NYN2022041501,escoe001
62,1,1,escoe001,21,BBCX,6/P6,NYN202204150,6,NYN2022041501,canor001
...,...,...,...,...,...,...,...,...,...,...
12773,3,1,canhm001,32,BCBBCX,S7/L7S,NYN202210050,S,NYN2022100501,guill001
12785,4,1,lindf001,22,BFBSX,43/G34D,NYN202210050,4,NYN2022100501,voged001
12795,5,1,canhm001,00,.X,53/G56,NYN202210050,5,NYN2022100501,guill001
12812,7,1,lindf001,02,.SSX,53/G6,NYN202210050,5,NYN2022100501,voged001


Add raw stats to each of the plays.

In [7]:
hitter_stats = hitter_plays.copy()
hitter_stats['tb'] = hitter_stats['result_char'].map({
    'S': 1,  # Single
    'D': 2,  # Double
    'T': 3,  # Triple
    'H': 4,  # Home run
}).fillna(0).astype(int)

# ob = on-base
hitter_stats['ob'] = hitter_stats['result_char'].map({
    'S': 1,  # Single
    'D': 1,  # Double
    'T': 1,  # Triple
    'H': 1,  # Home run
    'W': 1,  # Walk
    'I': 1,  # Intentional BB
    'P': 1,  # Hit by pitch
}).fillna(0).astype(int)

hitter_stats['ab'] = hitter_stats['result_char'].map({
    'W': 0,  # Walk, 1 base
    'I': 0,  # Intentional BB
    'P': 0,  # Hit by pitch
}).fillna(1).astype(int)

from IPython import display
display.display(hitter_stats)

,inning,inning_half,player_id,count_on_batter,pitches,result,game_id,result_char,game_team,next_batter,tb,ob,ab
58,1,1,mcnej002,22,CBFBX,S5/G56,NYN202204150,S,NYN2022041501,marts002,1,1,1
59,1,1,marts002,10,BX,S8/F8S.1-2,NYN202204150,S,NYN2022041501,lindf001,1,1,1
60,1,1,lindf001,02,CFX,46(1)/FO/G34.2-3;B-1,NYN202204150,4,NYN2022041501,alonp001,0,0,1
61,1,1,alonp001,10,*BX,7/SF/F7LSF/FL.3-H;1-2,NYN202204150,7,NYN2022041501,escoe001,0,0,1
62,1,1,escoe001,21,BBCX,6/P6,NYN202204150,6,NYN2022041501,canor001,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12773,3,1,canhm001,32,BCBBCX,S7/L7S,NYN202210050,S,NYN2022100501,guill001,1,1,1
12785,4,1,lindf001,22,BFBSX,43/G34D,NYN202210050,4,NYN2022100501,voged001,0,0,1
12795,5,1,canhm001,00,.X,53/G56,NYN202210050,5,NYN2022100501,guill001,0,0,1
12812,7,1,lindf001,02,.SSX,53/G6,NYN202210050,5,NYN2022100501,voged001,0,0,1


Calculate OPS for batters based on next batter ignoring those batter pairs with an insignificant number of plate appearances.

In [8]:
report = hitter_stats.copy()
report['name'] = report['player_id'].map(players)
report['next_name'] = report['next_batter'].map(players)

report = report.groupby(['player_id', 'name', 'next_batter', 'next_name']).agg(
    total_ob=('ob', 'sum'),
    total_pa=('result_char', 'count'),
    total_tb=('tb', 'sum'),
    total_ab=('ab', 'sum'),
)

# Filter out pairs with an insignificant number of plate appearances.
PA_FILTER = MIN_PA / 9
report = report[report.total_pa > (PA_FILTER)]

report['obp'] = (report.total_ob / report.total_pa).round(decimals=3)
report['slg'] = (report.total_tb / report.total_ab).round(decimals=3)
report['ops'] = (report.slg + report.obp).round(decimals=3)

report = report.sort_values(['player_id', 'ops'],ascending=False).groupby('player_id').head(8)

import math
from IPython.display import display, Markdown
display(
    Markdown(
        """
### {} {}: OPS of each player broken down by next batter
Notes:
1. Next batter is the actual player who batted next (i.e., not based on the lineup).

2. The last batter of the game for the team is not included. (Note that this raises OBP, SLG and OPS.)

3. The minimum number of plate appearances for the batter pair to be included in the report is {} (which is 1/9 of the min plate appearances to qualify for batting title).
        """.format(YEAR, TEAM, math.ceil(PA_FILTER))
    )
)
display(report)


### 2022 NYN: OPS of each player broken down by next batter
Notes:
1. Next batter is the actual player who batted next (i.e., not based on the lineup).

2. The last batter of the game for the team is not included. (Note that this raises OBP, SLG and OPS.)

3. The minimum number of plate appearances for the batter pair to be included in the report is 28 (which is 1/9 of the min plate appearances to qualify for batting title).
        

total_ob  total_pa  \
player_id name             next_batter next_name                              
nimmb001  Brandon Nimmo    lindf001    Francisco Lindor        28        74   
                           marts002    Starling Marte          63       191   
mcnej002  Jeff McNeil      alonp001    Pete Alonso             28        59   
                           canhm001    Mark Canha              17        38   
                           escoe001    Eduardo Escobar         28        78   
marts002  Starling Marte   lindf001    Francisco Lindor        82       229   
lindf001  Francisco Lindor mcnej002    Jeff McNeil             19        61   
                           alonp001    Pete Alonso             97       269   
escoe001  Eduardo Escobar  mcnej002    Jeff McNeil              8        29   
                           nidot001    Tomas Nido               9        30   
                           canhm001    Mark Canha              14        54   
                           guill001    Luis Guillorme           7        48   
canhm001  Mark Canha       lindf001    Francisco Lindor        13        32   
                           guill001    Luis Guillorme          19        47   
                           mcnej002    Jeff McNeil             11        45   
                           escoe001    Eduardo Escobar          9        37   
alonp001  Pete Alonso      mcnej002    Jeff McNeil             30        70   
                           voged001    Daniel Vogelbach        29        74   
                           escoe001    Eduardo Escobar         31        81   
                           canhm001    Mark Canha              10        29   
                           ruf-d001    Darin Ruf               10        30   

                                                         total_tb  total_ab  \
player_id name             next_batter next_name                              
nimmb001  Brandon Nimmo    lindf001    Francisco Lindor        23        61   
                           marts002    Starling Marte          71       174   
mcnej002  Jeff McNeil      alonp001    Pete Alonso             30        52   
                           canhm001    Mark Canha              15        33   
                           escoe001    Eduardo Escobar         32        69   
marts002  Starling Marte   lindf001    Francisco Lindor       113       211   
lindf001  Francisco Lindor mcnej002    Jeff McNeil             30        58   
                           alonp001    Pete Alonso             90       231   
escoe001  Eduardo Escobar  mcnej002    Jeff McNeil             14        26   
                           nidot001    Tomas Nido              13        28   
                           canhm001    Mark Canha              15        48   
                           guill001    Luis Guillorme           8        46   
canhm001  Mark Canha       lindf001    Francisco Lindor        14        27   
                           guill001    Luis Guillorme          15        38   
                           mcnej002    Jeff McNeil             13        42   
                           escoe001    Eduardo Escobar          8        35   
alonp001  Pete Alonso      mcnej002    Jeff McNeil             33        58   
                           voged001    Daniel Vogelbach        38        66   
                           escoe001    Eduardo Escobar         28        67   
                           canhm001    Mark Canha               8        25   
                           ruf-d001    Darin Ruf                5        25   

                                                           obp    slg    ops  
player_id name             next_batter next_name                              
nimmb001  Brandon Nimmo    lindf001    Francisco Lindor  0.378  0.377  0.755  
                           marts002    Starling Marte    0.330  0.408  0.738  
mcnej002  Jeff McNeil      alonp001    Pete Alonso       0.475  0.577  1.052  
                          